In [1]:
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings("ignore", category=FutureWarning)

import pandas as pd
import nltk

from nlu_engine import NLUEngine
from nlu_engine import MacroDataRefinement
from nlu_engine import MacroEntityRefinement

from nlu_engine import DataUtils
from nlu_engine import RenderJSON

from nlu_engine import Analytics

from nlu_engine import EntityExtractor, crf

# Macro NLU Data Refinement

It's a bit like the TV show [Serverance](https://www.imdb.com/title/tt11280740/) .

![Helly R and Mark S](https://media.npr.org/assets/img/2022/02/15/atv_severance_photo_010103-5f8033cc2b219ba64fe265ce893eae4c90e83896-s1100-c50.jpg "Helly R and Mark G")

*Helly R*: `My job is to scroll through the spreadsheet and look for the numbers that feel scary?`

*Mark S*: `I told you, you’ll understand when you see it, so just be patient.`

![MDR](https://www.imore.com/sites/imore.com/files/styles/large/public/field/image/2022/03/refinement-software-severance-apple-tv.jpg "serverance micro data refinement")

*Helly R*: `That was scary. The numbers were scary.`

Hopefully the intents and entities that are wrong aren't scary, just a bit frustrating. Let's see if we can find the right ones.

NOTE: We will use Logistic Regression with TFIDF features to train our intent models and CRFs for entity exraction. Why? Well, they are very fast and both methods aren't state-of-the-art. This is good, because it is easier to find problems we will need to refine in the dataset than if we were to use a proper NLU engine like Snips or something SOTA like BERT. It is very important to note that some of the the problems we will pick up on, might not be an actual issue, but might be due to the limitations of the models. Refining the real problems and ignoring the limitations of the models is a good way to improve the models. Then when the dataset is ready, we can use some more advanced NLU engine and get the best performance possible.

* Macro NLU Data Refinement: Intent
* Macro NLU Data Refinement: Entity


Load the dataset

In [2]:
try:
    nlu_data_df = pd.read_csv(
        'data/refined/nlu_data_refined_df.csv', sep=',', index_col=0)
    print('Successfully loaded nlu_data_refined_df.csv')
except:
    data = 'data/NLU-Data-Home-Domain-Annotated-All-Cleaned.csv'
    nlu_data_df = DataUtils.load_data(
    data
)

Successfully loaded nlu_data_refined_df.csv


Make sure `nlu_data_df['answer_normalised']` is taken from `nlu_data_df['answer_annotation']`

In [3]:
nlu_data_df = DataUtils.convert_annotated_utterances_to_normalised_utterances(
    nlu_data_df)

In [25]:
nlu_data_df

,userid,answerid,notes,question,suggested_entities,answer,answer_normalised,scenario,intent,predicted_label,intent_refined,entity_refined,remove,status,answer_annotation
0,1.0,1.0,NaN,Write what you would tell your PDA in the foll...,"date, time",wake me up at 5am this week,wake me up at five am this week,alarm,alarm_set,alarm_set,NaN,NaN,NaN,NaN,wake me up at [time : five am] [date : this week]
1,1.0,2.0,NaN,Write what you would tell your PDA in the foll...,"date, time",wake me up at 9am on Friday,wake me up at nine am on friday,alarm,alarm_set,alarm_set,NaN,NaN,NaN,NaN,wake me up at [time : nine am] on [date : friday]
2,1.0,3.0,NaN,Write what you would tell your PDA in the foll...,"date, time",set an alarm for two hours from now,set an alarm for two hours from now,alarm,alarm_set,alarm_set,NaN,NaN,NaN,NaN,set an alarm for [time : two hours from now]
3,1.0,31.0,NaN,Write what you would tell your PDA in the foll...,player_setting,Olly quiet!,quiet,audio,volume_mute,volume_mute,NaN,NaN,NaN,NaN,quiet
5,1.0,33.0,NaN,Write what you would tell your PDA in the foll...,player_setting,Olly pause for ten seconds,pause for ten seconds,audio,volume_mute,volume_mute,True,NaN,True,NaN,pause for [time : ten seconds]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25711,NaN,781.0,NaN,NaN,NaN,"that's cool, musch appreciated, olly.","that's cool, musch appreciated, olly.",general,praise,praise,NaN,NaN,NaN,NaN,"that's cool, musch appreciated, olly."
25712,NaN,782.0,NaN,NaN,NaN,"you are hero, appreciated.","you are hero, appreciated.",general,praise,praise,NaN,NaN,NaN,NaN,"you are hero, appreciated."
25713,NaN,783.0,NaN,NaN,NaN,"thanks, that's nice.","thanks, that's nice.",general,praise,praise,NaN,NaN,NaN,NaN,"thanks, that's nice."
25714,NaN,784.0,NaN,NaN,NaN,"that's cool, thank you so much.","that's cool, thank you so much.",general,praise,praise,NaN,NaN,NaN,NaN,"that's cool, thank you so much."


In [ ]:
#TODO: does this NEED to be here, LOL!
from matplotlib.pyplot import axis

We should remove the unwanted entries for the next few steps.

In [4]:
removed_nlu_data_refined_df = nlu_data_df[nlu_data_df['remove'] != True]

# Entity extraction reports

The entity extraction could be greatly improved by improving the features it uses. It would be great if someone would take a look at this. Perhaps the CRF features similar to what Snips uses would be better such as Brown clustering (probably).

In [ ]:
#TODO: implement brown clustering to improve entity extraction (see entity_extractor.py)

It is important to have the NLTK tokenizer to be able to extract entities.

In [ ]:
try:
    nltk.data.find('tokenizers/punkt')
except LookupError:
        nltk.download('punkt')

Due to this error featured in [this git issue](https://github.com/TeamHG-Memex/sklearn-crfsuite/issues/60) we have to use an older version of scikit learn (sklearn<0.24), otherwise the latest version would work. Hopefully this gets fixed one day..

In [ ]:
entity_report_df = NLUEngine.evaluate_entity_classifier(
    data_df=removed_nlu_data_refined_df, cv=4)

In [ ]:
entity_report_df

Let's have a look at the report plotted for us

In [ ]:
Analytics.plot_report(entity_report_df)

We will need to extract out the entity types from the dataset. We need this to calculate the precentage of entities, to filter out any domains that have too few entities for benchmarking by domain instead of all domains together.

In [5]:
removed_nlu_data_refined_df['entities'] = removed_nlu_data_refined_df['answer_annotation'].apply(
    EntityExtractor.extract_entities)

/home/bartmoss/code/NLU-engine-prototype-benchmarks/.venv/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [6]:
removed_nlu_data_refined_df['entity_types'] = removed_nlu_data_refined_df['entities'].apply(
    EntityExtractor.extract_entity_types)

/home/bartmoss/code/NLU-engine-prototype-benchmarks/.venv/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [ ]:
#TODO: add this as a method to the entity extractor class (should I still do this? YES!)

In [7]:
import numpy as np
removed_nlu_data_refined_df['entity_types'] = removed_nlu_data_refined_df['entity_types'].apply(lambda y: np.nan if len(y)==0 else y)
removed_nlu_data_refined_df['entities'] = removed_nlu_data_refined_df['entities'].apply(lambda y: np.nan if len(y)==0 else y)

/home/bartmoss/code/NLU-engine-prototype-benchmarks/.venv/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/home/bartmoss/code/NLU-engine-prototype-benchmarks/.venv/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [ ]:
removed_nlu_data_refined_df

In [ ]:
domain_entity_reports_df = NLUEngine.get_entity_reports_for_domains(removed_nlu_data_refined_df)

In [ ]:
domain_entity_reports_df

In [ ]:
domain_entity_reports_df = domain_entity_reports_df[domain_entity_reports_df['entity-type'].str.contains(
    'weighted avg')].sort_values(by='f1-score', ascending=True)

In [ ]:
domain_entity_reports_df

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
#TODO: rename plot_report to plot_intent_report and this code below should be a method called plot_entity_report
label = domain_entity_reports_df.columns[5]

fig, ax = plt.subplots()
graph_report_df = domain_entity_reports_df.sort_values(
            by='f1-score', ascending=True)
y_axis = np.arange(len(graph_report_df[label]))

ax.barh(y_axis, graph_report_df['f1-score'],
        align='center', color='b')

fig.set_figheight(16)
fig.set_figwidth(12)

ax.set_title(f'f1-scores by {label}', fontsize=24)
ax.set_xlabel("f1-score", fontsize=18)
ax.tick_params(axis='x', labelsize=18)
ax.set_ylabel(label, fontsize=18)
ax.set_yticks(y_axis, graph_report_df[label], fontsize=16)
fig.tight_layout()

## entity classification 

We can get the `predicted_tagging` and find all of the entities that are being incorrectly classified.

If you don't already have a saved model (for the default data set there is already one), you can train a model and save it.

In [ ]:
crf_model = NLUEngine.train_entity_classifier(removed_nlu_data_refined_df)

In [ ]:
model_path = 'models/analytics/entity_tagger.sav'

In [ ]:
DataUtils.pickle_model(classifier=crf_model, model_path=model_path)

You can open it from here.

In [8]:
model_path = 'models/analytics/entity_tagger.sav'

In [9]:
crf_model = DataUtils.import_pickled_model(model_path)

Now, let's run it across our data set to get the `predicted_tagging`.

In [10]:
removed_nlu_data_refined_df['predicted_tagging'] = removed_nlu_data_refined_df['answer_normalised'].apply(
    lambda x: NLUEngine.create_entity_tagged_utterance(x, crf_model))

/home/bartmoss/code/NLU-engine-prototype-benchmarks/.venv/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


We can now load a dataset of all of the incorrectly predicted entities.

In [ ]:
#TODO: I might move this out since it is only being done after the overlapping entities have been removed
incorrect_predicted_entities_df = removed_nlu_data_refined_df[removed_nlu_data_refined_df['answer_annotation']
                            != removed_nlu_data_refined_df['predicted_tagging']]

In [ ]:
incorrect_predicted_entities_df

# Domain specific entity analysis

For a new domain start here!

Pick a domain to start.

In [11]:
domain_selection = MacroDataRefinement.list_and_select_domain(
    removed_nlu_data_refined_df)

In [12]:
domain_df = DataUtils.get_domain_df(
    removed_nlu_data_refined_df, domain_selection)

Successfully loaded dataframe


In [ ]:
domain_df

Incorrectly predicted entities report

In [ ]:
incorrect_predicted_entities_report = MacroEntityRefinement.get_incorrect_predicted_entities_report(
    domain_df, entity_report_df, domain_entity_reports_df)

#TODO: Have user go through general refinement in batches of 100 or less per domain


Let's have a better look at this report of incorrectly predicted entities.

In [ ]:
RenderJSON(incorrect_predicted_entities_report)

## Entity overlap refinement
We want to find the entities that have overlapping entity types. Entities (the words themselves that get tagged), should very rarely have more than one entity type in a domain. If they do, then we need to refine the dataset.

take the `domain_df['entities']` and get a list of dictionaries for:
* `id`: the original index of the row
* `entity_type`: the entity type
* `entity_words`: the joined words with spaces that make up the entity

and then drop it into a dataframe

In [13]:
correct_entity_types, incorrect_entity_types, overlapping_entity_words = MacroEntityRefinement.pick_correct_entity_type_for_overlapping_entities(
    domain_df)

We have the `correct_entity_types`. We will need those to know what to change the `entity_type` to.

In [ ]:
correct_entity_types

We should probably get a list of the `incorrect_entity_types` to go along with our `correct_entity_types`, so we know what to replace.

In [ ]:
incorrect_entity_types

How about we for loop over the zip of `incorrect_entity_types` and `correct_entity_types` and replace the `entity_type` in `overlapping_domain_df` with the `correct_entity_type`.

In [14]:
refined_overlapping_domain_df = MacroEntityRefinement.refine_overlapping_entity_types(
    domain_df, correct_entity_types, incorrect_entity_types, overlapping_entity_words)

replacing: timeofday : every morning
with: general_frequency : every morning
replacing: alarm_type : soccer practice
with: event_name : soccer practice


## Entity refinement
We will want to refine all of the entries that have the incorrect entity types.

In [15]:
domain_incorrect_predicted_entities_df = refined_overlapping_domain_df[refined_overlapping_domain_df['answer_annotation']
                                                              != refined_overlapping_domain_df['predicted_tagging']]


In [16]:
to_review_sheet = MacroDataRefinement.create_sheet(
    domain_incorrect_predicted_entities_df)
to_review_sheet

/home/bartmoss/code/NLU-engine-prototype-benchmarks/.venv/lib/python3.7/site-packages/pandas/core/frame.py:4913: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


Sheet(cells=(Cell(column_end=0, column_start=0, numeric_format=None, row_end=43, row_start=0, squeeze_row=Fals…

In [ ]:
# TODO: start from here, refine then save to CSV the changes. Repeat until all domains have been refined.

In [17]:
reviewed_domain_entities_df = MacroDataRefinement.convert_sheet_to_dataframe(to_review_sheet)

In [19]:
refined_domain_entities_df = reviewed_domain_entities_df.apply(
    MacroDataRefinement.replace_annotated_utterance_with_predicted_tagging, axis=1)

In [23]:
refined_domain_entities_df.to_csv(
    f'data/refined/entities/{domain_selection}_refined_entities_df.csv')

In [24]:
refined_domain_entities_df = MacroDataRefinement.mark_entries_as_refined(
    refined_dataframe=refined_domain_entities_df, refined_type='entity')

In [31]:
removed_nlu_data_refined_df.update(refined_domain_entities_df)

In [34]:
removed_nlu_data_refined_df.to_csv(
    f'data/removed_nlu_data_refined_updated.csv')

In [ ]:
# FROM HERE: loop back for each domain to refine the entities until done.

In [ ]:
# TODO: once all of the refinements of each domain have been completed the nlu_data_refined_df should be updated with the refined data from removed_nlu_data_refined_df!

In [ ]:
#TODO: refactor intent refinement to dump the csvs into a sub dir called intents!

In [ ]:
#TODO: It is probably a good idea to drop all of the ones that lack a good support.
#NOTE: But it didn't work to fix the problem.
remove_entities = [
    'music_album',
    'game_type',
    
]
removed_nlu_data_refined__entities_cleaned_df = removed_nlu_data_refined_df[~removed_nlu_data_refined_df['answer_annotation'].str.contains('|'.join(remove_entities))]

In [ ]:
Analytics.plot_report(entity_report_df)

In [ ]:
#TODO: Remove/replace worst: add in state features like here: https://sklearn-crfsuite.readthedocs.io/en/latest/tutorial.html#let-s-check-what-classifier-learned
# Specifically, we want print_state_features()

In [ ]:
def print_state_features(state_features):
    for (attr, label), weight in state_features:
        print("%0.6f %-8s %s" % (weight, label, attr))


In [ ]:
from nlu_engine import crf
from collections import Counter

In [ ]:
crf_model.state_features_

In [ ]:
print_state_features(Counter(crf.state_features_).most_common(100))

In [ ]:
# TODO: review the most common features, none of them are the word parts (chunks) or POS tags, are these even needed or helpful?
# Can we remove them and speed up the process?
# What other features could be used? Word2vec? Brown clustering?

In [ ]:
domain_selection = MacroDataRefinement.list_and_select_domain(nlu_data_df)

As we have seen from the entity extraction report, the entity extraction is not working for the alarm_type.

In [ ]:
#TODO: review all scoring 0, see if they can be completely dropped or what
entity_to_refine = 'alarm_type'
nlu_scenario_df = removed_nlu_data_refined_df[removed_nlu_data_refined_df['answer_annotation'].str.contains(
    entity_to_refine)]

In [ ]:
nlu_scenario_df

In [ ]:
def remove_entity(df, entity_to_remove):
    """
        Remove all entries of an entity type from the dataframe.
        :param df: pandas dataframe
        :return: pandas dataframe
        """
    updated_df = df.copy()
    updated_df.loc[updated_df['answer_annotation'].str.contains(
        entity_to_remove), 'remove'] = True
    return updated_df


In [ ]:
updated_df = remove_entity(removed_nlu_data_refined_df, entity_to_refine)

In [ ]:
removed_nlu_data_refined_df[removed_nlu_data_refined_df['answer_annotation'].str.contains(
    entity_to_refine)]

## Entity Convert to ipysheet and review
TODO: add in description of the types of fixes we can do to the NLU data for entity


In [ ]:

# TODO: same as above for intents but with predicted entities: report on them, break them down into a dictionary of dataframes and refine them..

For the example with 'alarm' and the alarm_type:
* We see that the alarm_type entities are really event_name(ie wake up, soccer practice) except for ID 5879, we will need to change them to event_name and remove ID 5879.
* The last one(ID 6320) is a mistake. Someone got confused with the prompt and assumed alarm is a security system. This is out of scope for the alarm domain, as the alarms are ones set on a phone or other device. We will drop this utterance.
Once you are done reviewing, you convert it back to a dataframe and check to make sure it looks okay.
Let's change all alarm_type entities to event_name.

In [ ]:

reviewed_scenario_df['answer_annotation'] = reviewed_scenario_df['answer_annotation'].str.replace(
    'alarm_type', 'event_name')
reviewed_scenario_df


Okay dokey, now we can merge this with the original data set and see if it made a difference already(well of course it did!).

In [ ]:
nlu_data_df.drop(
    reviewed_scenario_df[reviewed_scenario_df['remove'] == True].index, inplace=True)

reviewed_scenario_df = reviewed_scenario_df[~reviewed_scenario_df['remove'] == True]

nlu_data_df.loc[nlu_data_df.index.intersection(
    reviewed_scenario_df.index), 'answer_annotation'] = reviewed_scenario_df['answer_annotation']

nlu_data_df[(nlu_data_df['scenario'].str.contains('alarm')) & (nlu_data_df['answer_annotation'].str.contains(
    'event_name'))]


### Benchmark changed data set
TODO: repeat reports for the changed data set for domain and entities and compare


In [ ]:

entity_reviewed_report_df = NLUEngine.evaluate_entity_classifier(
    data_df=nlu_data_df)
entity_reviewed_report_df.sort_values(by=['f1-score'])

If you are sure it is okay, you can save it as a csv file, make sure to name it correctly(i.e. `alarm_domain_first_review.csv`)

In [ ]:
reviewed_scenario_df.to_csv('alarm_domain_first_review.csv')

Load it back up and check to make sure it looks okay. Make sure to give it the right name!


In [ ]:
audio_domain_first_review_df = pd.read_csv(
    'alarm_domain_first_review.csv', index_col=0)
audio_domain_first_review_df.tail(50)


In [ ]:
# TODO: implement the evaluate_classifier in the NLU engine to check f1 score for intents and entities in the domain vs original NLU data of domain!
# Value: benchmark!
#TODO: implement a flow for getting the domains with the lowest f1 scores by intent/domain and entities and cleaning them by the order of the lowest f1 scores
# TODO: concat all reviewed dfs and save to csv
# TODO: add benchmark for whole NLU data set before and after cleaning! (by intents and domains!)
# TODO: review the review marked entries
# TODO: add new column for notes
# TODO: change flow of review for only ones that should be reviewed, not all of the ones that have been changed (track changes by comparing against the original data set)
# TODO: do the changed utterances have to be changed in other fields too or is it just enough for the tagged utterancve field?
# TODO: add visualizations of domains, their intents, keywords in utterances, and entities to top
